## Dataset creation

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Gen AI/Sample_data/Myntra Fasion Clothing.csv')

<ipython-input-2-24d6482837e5>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Gen AI/Sample_data/Myntra Fasion Clothing.csv')


In [ ]:
df.columns

Index(['URL', 'Product_id', 'BrandName', 'Category', 'Individual_category',
       'category_by_Gender', 'Description', 'DiscountPrice (in Rs)',
       'OriginalPrice (in Rs)', 'DiscountOffer', 'SizeOption', 'Ratings',
       'Reviews'],
      dtype='object')

In [ ]:
df.columns

cols = ['BrandName', 'Category', 'Individual_category',
       'category_by_Gender', 'Description', 'DiscountPrice (in Rs)',
       'DiscountOffer', 'SizeOption', 'Ratings'
       ]

In [ ]:
x = ''
for i in cols:
  y= f"{i}: "+"{row['" + i+"']} "
  x+=y

In [ ]:
x

"BrandName: {row['BrandName']} Category: {row['Category']} Individual_category: {row['Individual_category']} category_by_Gender: {row['category_by_Gender']} Description: {row['Description']} DiscountPrice (in Rs): {row['DiscountPrice (in Rs)']} DiscountOffer: {row['DiscountOffer']} SizeOption: {row['SizeOption']} Ratings: {row['Ratings']} "

In [ ]:
df['Description'][120]
df['URL'][1260]


'https://www.myntra.com/jeans/mast--harbour/mast--harbour-men-navy-blue-slim-tapered-fit-mid-rise-clean-look-stretchable-jeans/12983716/buy'

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# %%capture
df['combined'] = ''
for i,row in tqdm(df.iterrows()):
  df.loc[i,'combined'] = f"""BrandName: {row['BrandName']} Category: {row['Category']} {row['Individual_category']} Gender: {row['category_by_Gender']} Description: {row['Description']} Price: {row['DiscountPrice (in Rs)']} SizeOption: {row['SizeOption']} Ratings: {row['Ratings']}   """
  # print(i,x)
  # break

526564it [02:05, 4185.51it/s]


In [ ]:
df2 = df[['URL', 'Product_id','combined']]

In [ ]:
document = ""
doc_list = []
for i,row in tqdm(df2.iterrows()):
  chunk = f"**Product {row['Product_id']}**.\n{row['combined']}\n\n"
  document +=  chunk
  doc_list.append(chunk)


526564it [00:55, 9464.50it/s]


In [ ]:
with open('myntra_fashion.txt', 'w') as file:
    file.write(document)

## Load LLM

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-38bjk5bu/unsloth_6c85a8ff6be847a68061fd0cbc7937ff
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-38bjk5bu/unsloth_6c85a8ff6be847a68061fd0cbc7937ff
  Resolved https://github.com/unslothai/unsloth.git to commit e41cc4093c70095e4aef390c8afae85c38aa4eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# import os
# from google.colab import userdata

# max_seq_length = 2048
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models unsloth supports
# fourbit_models = [
#     #"unsloth/mistral-7b-v0.3-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     # "unsloth/llama-3-8b-bnb-4bit",
#     #"unsloth/llama-3-8b-Instruct-bnb-4bit",
#     #"unsloth/llama-3-70b-bnb-4bit",
#     #unsloth/Phi-3-mini-4k-instruct",
#     #"unsloth/Phi-3-medium-4k-instruct",
#     #"unsloth/mistral-7b-bnb-4bit",
#     #"unsloth/gemma-7b-bnb-4bit",
# ]

# # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/llama-3-8b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     token = userdata.get("HF_TOKEN")
# )

## Qdrant Vector DB

In [ ]:
%%capture
%pip install llama-index
%pip install llama-index-vector-stores-qdrant
%pip install llama-index-vector-stores-chroma
%pip install llama-index chromadb
%pip install llama-index-readers-file
%pip install llama-index-llms-openai
%pip install qdrant_client
%pip install chromadb
%pip install llama-index-embeddings-fastembed
%pip install llama-index-embeddings-huggingface

In [ ]:
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client
import os
from google.colab import userdata


In [ ]:
from llama_index.core import Document, VectorStoreIndex

documents = [Document(text=t) for t in doc_list]

In [ ]:
client = qdrant_client.QdrantClient(
    url = "https://e1e16ea6-1c74-43be-a4a3-8564d87a65ed.us-east4-0.gcp.cloud.qdrant.io",
    api_key = userdata.get("QDRANT_API_KEY")
)

In [ ]:
pip install --upgrade huggingface-hub==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.23.0 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="mistralai/Mistral-7B-Instruct-v0.3"
# )
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# documents = SimpleDirectoryReader("/content/drive/MyDrive/Gen AI/Sample_data/myntra").load_data()


In [ ]:
vector_store = QdrantVectorStore(client = client, collection_name = "fashion_data2",)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex(documents, storage_context = storage_context, embed_model = embed_model)

In [ ]:
index

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("myntra_fashion")

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)